In [1]:
import matplotlib
from matplotlib import pyplot as plt
from numpy import *
import numpy as np
import random
import scipy.io
import sklearn.utils

f1 = scipy.io.loadmat('train.mat')
%matplotlib inline

In [141]:
#Input Layer 28x28x1
#Convolution Layer
#Output Layer 10x1
#10 filters, 3x3x1


def reshape_x(x): #add bias row to NN input
#     X = np.reshape(x,(len(x),len(x.T)**2))
    X = x/np.max(x)
    return x

def reshape_y(y,outputSize,filter_Count): #change y from a single value to an array of 10 values corresponding to output 
    tempy = np.zeros((len(y),outputSize))
    for i,r in enumerate(y):
        tempy[i][r[0]] = 1
    tempy = np.repeat(tempy,filter_Count,axis=0)
    return tempy


class NeuralNet(): #single convNet with a single fully connected neural layer, pool layer?
    def __init__(self,input_Size,filter_Count,filtered_Size,hidden_Size,output_Size): #add bias later
        self.filter_Count = filter_Count
        self.input_Size = input_Size
        self.filtered_Size = filtered_Size
        self.hidden_Size = hidden_Size
        self.output_Size = output_Size

        filter_Weights = np.random.normal(0,0.1,input_Size*filter_Count)
        self.filter_Weights = np.reshape(filter_Weights,(filter_Count,input_Size)).T

        V = np.random.normal(0,0.001,hidden_Size*(filtered_Size))
        self.V = np.reshape(V,(hidden_Size,filtered_Size)).T

        W = np.random.normal(0,0.001,output_Size*(hidden_Size))
        self.W = np.reshape(W,(output_Size,hidden_Size)).T
        
    def forward(self, inputs):
        filter_train = self.filter_Weights
        
        self.conv_Vals = np.dot(filter_train.T,inputs) # FX
        self.activation_conv = self.activation_fnc1(self.conv_Vals) # f(FX)

        self.hidden_Vals = np.dot(self.activation_conv,self.V) # Vf(FX)
        self.activation_H = self.activation_fnc2(self.hidden_Vals) # f(Vf(FX))
   
        
        self.pre_guess = np.dot(self.activation_H,self.W)  # Wf(Vf(FX))
        self.guess = self.activation_fnc3(self.pre_guess)  # f(Wf(Vf(FX)))
        
        
        
    def cross(self,inputs,outputs):
        self.forward(inputs)
        return -sum((outputs*np.log(self.guess)+(1-outputs)*np.log(1-self.guess)))
    
    
    def crossBack(self,inputs,outputs):
       
        self.forward(inputs) # f(Wf(Vf(FX)))
        
        back1 = (-(outputs/self.guess)+(1-outputs)/(1-self.guess))*self.activation_fnc1(self.pre_guess,True) #(-y+y_h)f'(Wf(Vf(FX)))
        dW = np.dot(self.activation_H.T,back1) # (-y+y_h)f'(Wf(Vf(FX))) f(Vf(FX))
        
        back2 = np.dot(back1,self.W.T)*self.activation_fnc2(self.hidden_Vals,True) #(-y+y_h)f'(Wf(Vf(FX))) W  f'(Vf(FX))
        dV = np.dot(self.activation_conv.T,back2) #(-y+y_h)f'(Wf(Vf(FX))) W  f'(Vf(FX)) f(FX)
        
        back3 = np.dot(back2,self.V.T)*self.activation_fnc3(self.conv_Vals,True) #(-y+y_h)f'(Wf(Vf(FX))) W  f'(Vf(FX)) V f'(FX)
        dF = np.dot(inputs,back3.T) #(-y+y_h)f'(Wf(Vf(FX))) W  f'(Vf(FX)) V f'(FX) X
        return dF, dV, dW
        
        
    def update_Weights(self, updates, step_size):
        self.filter_Weights -= updates[0]*step_size
        self.V -= updates[1]*step_size
        self.W -= updates[2]*step_size
        
        
    def activation_fnc1(self, z, deriv=False): #currently sigmoid
        a = None
        if deriv:
            a = (1-(1+np.exp(-z))**-1)/(1+np.exp(-z))
        else:
            a = (1+np.exp(-z))**-1
        nans = isnan(a)
        a[nans] = 0
        return a
    
    def activation_fnc2(self, z, deriv=False): #currently tanh
        a = None
        if deriv:
            a = 1-np.tanh(z)**2
        else:
            a = np.tanh(z)
        nans = isnan(a)
        a[nans] = 0
        return a
    
    def activation_fnc3(self, z, deriv=False): #currently tanh
        a = None
        if deriv:
            a = (1-(1+np.exp(-z))**-1)/(1+np.exp(-z))
        else:
            a = (1+np.exp(-z))**-1
        nans = isnan(a)
        a[nans] = 0
        return a
    
    def train(self, inputs, outputs, iters, feat_size,e):
        self.inputs = inputs
        self.outputs = outputs
        
        for i in range(iters):
            r = random.randint(0,len(inputs)-1)
            dp = im2col(inputs[r],feat_size,feat_size) #change to vars
#             dp = np.array([inputs[r]])
            dpl = np.array([self.outputs[r]])
            dpl = reshape_y(dpl,10,self.filter_Count)
            deriv = self.crossBack(dp,dpl)
            if (i+1) % 1000 == 0:
                count = 0
                for i in range(100):
                    r = random.randint(0,len(inputs)-1)
                    d = im2col(inputs[r],feat_size,feat_size)
                    a = np.array([self.outputs[r]])
                    a = reshape_y(a,10,self.filter_Count)
                    count += self.cross(d,a)
                print(count)
            self.update_Weights(deriv,e)
    
    
    def predict(self,inputs,box):
        dp = im2col(inputs,box,box) #change to vars
        self.forward(dp)
#         self.forward(inputs)
        return self.guess
    

In [142]:
def im2col(matrix,box_width,box_height,stride=1): #need to fix stride
    height = len(matrix)
    width = len(matrix.T)

    ret_height = box_width*box_height
    ret_width = int(((width-box_width)/stride+1)**2)
    
    
    im_matrix = np.zeros((1,box_width*box_height))
    for i in range(height):
        for j in range(width):
            if (j+box_width <= width) and (i+box_height <= height):
                im_matrix = np.append(im_matrix,
                                      matrix[i:i+box_height,j:j+box_width].reshape(box_width*box_height))
            else:
                break
    val_matrix = im_matrix.T[box_width*box_height:]
    ret_matrix = val_matrix.reshape((ret_width,ret_height)).T
    return ret_matrix
    

In [143]:
y = f1['train_labels']
X = np.swapaxes(f1['train_images'],1,0).T

X = reshape_x(X)
X,y = sklearn.utils.shuffle(X,y,random_state=0)

In [172]:
box = 9
CrossNN = NeuralNet(box**2,30,400,300,10)

In [ ]:
CrossNN.train(X,y,20000,box,np.exp(-15))

1193.03180961
1022.35892579
977.477133661
1219.21766438
1388.66660426
851.387219063
1161.1742175
1020.9458283
600.456177218
1032.54390096
1565.03540163
1112.23859655
768.554004484
1785.64353236
989.3613204
1392.13682191

In [ ]:
h = 0
for i in range(100):
    r = random.randint(0,len(X)-1001)
    h += int(np.argmax(np.average(CrossNN.predict(X[i+r],box),axis=0))==y[i+r])
print(h/100)

In [ ]:
plt.imshow(np.dot(CrossNN.filter_Weights.T[10].reshape(1,box**2),im2col(X[1056].reshape(28,28),box,box)).reshape(24,24))

In [ ]:
plt.imshow(np.dot(CrossNN.filter_Weights.T[29].reshape(1,box**2),im2col(X[1056].reshape(28,28),box,box)).reshape(24,24))

In [ ]:
arr = np.dot(CrossNN.filter_Weights.reshape(30,25),im2col(X[1025],box,box)).reshape(30,24,24)
arr2 = np.array(arr[0])
for i in range(1,30):
    arr2 = np.hstack((arr2,arr[i]))
plt.imshow(arr2)
print(arr2.shape)